# ПОИСК ПОХОЖИХ ИНТЕНТОВ

Данильченко Вадим

In [8]:
import numpy as np
import pandas as pd
import time
from datetime import datetime as dt
import joblib
import pymorphy2, re
from string import punctuation
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import warnings
from tqdm import trange
from keras.preprocessing.text import text_to_word_sequence
import string

warnings.filterwarnings('ignore')

------------------------------------------------------------------------------------------------------------------------------

если в выгрузке уже есть колонка отпрепроцешенных фраз, то идем к пункту 2, иначе начинаем отсюда

### 1. ПОДГОТОВИМ ВЫГРУЗКУ

In [287]:
output_data=pd.read_csv('data_history.csv', encoding='cp1251', delimiter='\t')

In [293]:
data_apr = pd.read_csv('04-2020.txt', encoding='cp1251', delimiter='\t')
data_apr.shape

(146826, 24)

In [294]:
output_data = pd.concat([output_data, data_apr], axis=0)
output_data.shape

(1463957, 24)

### зададим название поля, содержащее текст сообщений

в таблице [diamch.reports_intent_score] это поле "new_message"

In [296]:
message_col='Сообщение'

### Лемматизируем текст и исключим некоторые опечатки

In [24]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [20]:
tokenizer = Tokenizer()

In [32]:
text_to_word_sequence('привет каКк дела 123ё?')

['привет', 'какк', 'дела', '123ё']

In [297]:
morph = pymorphy2.MorphAnalyzer()

# nltk.download("stopwords")

# подгрузим стоп-слова
try:
    russian_stopwords = stopwords.words("russian")
except:
    russian_stopwords=joblib.load('chat-bot themes materials/russian_stopwords.pkl')


stop_words=['нахер', 'третье', 'кола', 'жаль','который', 'кпникула' , 'мегодень', 'каедомлентить', 'точнуб', 'закасить',\
           'лайна', 'вслехать', 'нат', 'кроме', 'срочно', 'ередить', 'блчдь', 'могуля', 'пиздец', 'копа', 'бэкома', 'ирек',\
           'каклть', 'хрень', 'нахер', 'таки', 'никто', 'муха', 'сраный', 'туда', 'слушай', 'долдно', 'каков', 'как', 'какой',\
           'блядь', 'горелый', 'можито', 'пора', 'недолжный', 'это', 'ваш' , 'nan', 'косасия' ,'пидар', 'пидра', 'понос' ,\
            'отъестись', 'нибыть', 'вообще', 'видимо', 'позвать', 'назад', 'иметь', 'ввиду', 'добрый']

add2stop_words=['подскажите', 'подсказать', 'пожалуйста', 'необходимо', 'здравствуйте', 'скажите', 'пожалуйста',
                'хочу', 'почему', 'привет']

russian_stopwords=russian_stopwords + stop_words + add2stop_words

# исключим из словаря некоторые слова
excl_from_dict=['как', 'какой', 'иметь', 'который', 'какая', 'нет', 'было', 'быть', 'ничего',
                'где', 'куда', 'зачем', 'не', 'можно']

russian_stopwords=list(set([i for i in russian_stopwords if i not in excl_from_dict]))

# список приветственных словосочетаний, которые исключим из предложения
del_from_sentence=['добрый день', 'добрый вечер', 'просьба подсказать', 'доброго времени', 'доброй ночи', 'доброе время суток', \
                   'день добрый', 'доброго времени', 'доброго дня', 'вечер добрый', 'утро доброе', 'подскажите пожалуйста']

# скорректируем опечатки
errors=joblib.load('chat-bot themes materials/errors2correct_dict.pkl')

def preprocess_text(text):
    text = str(text).lower().strip()
    text = text.replace("ё", "е")
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
    text = re.sub('@[^\s]+', 'USER', text)
    text = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', text) 
#     text = re.sub(' +', ' ', text)
    text = re.sub(r'[^\w\s]+|[\d]+', ' ',text)
    obj = re.match(r'[\d+\W+]{,}', text)
    text = text[0 if obj is None else len(obj.group(0)):]    
    text = re.sub(r'[\s+]', ' ', text)
    text = re.sub('([a-zа-яёе])\1{2,}', r'\1\1', text)  # aaaaa -> aa
    text = re.sub(r'([^a-zа-яёе0-9])\1{1,}', r'\1', text)  # )))) -> )
    tokens = text_to_word_sequence(text)
    tokens_result=[]
    for token in tokens:
        if len(token.strip())>0 and token.strip() not in russian_stopwords:
            token=morph.parse(token.strip())[0].normal_form
            if token in errors.keys():
                tokens_result.append(errors[token])
            else:
                tokens_result.append(token)
            
#               and token.strip() not in punctuation]
    
    return " ".join(tokens_result)

In [298]:
start_time = time.time()

# индекс столбца в списке названий полей
message_col_ind=output_data_tmp.columns.tolist().index(message_col)

# запишем в список предложения
sentences=[]
for i in trange(output_data_tmp.shape[0]):
    sentences.append(preprocess_text(output_data_tmp.iloc[i,message_col_ind]))

print("--- %s minutes ---" % (round((time.time() - start_time)/60)))
sentences[:2]



  0%|          | 0/919652 [00:00<?, ?it/s]

  0%|          | 69/919652 [00:00<22:13, 689.80it/s]

  0%|          | 208/919652 [00:00<18:51, 812.51it/s]

  0%|          | 345/919652 [00:00<16:34, 924.57it/s]

  0%|          | 480/919652 [00:00<15:01, 1019.87it/s]

  0%|          | 618/919652 [00:00<13:50, 1106.22it/s]

  0%|          | 761/919652 [00:00<12:54, 1186.35it/s]

  0%|          | 914/919652 [00:00<12:02, 1270.80it/s]

  0%|          | 1099/919652 [00:00<10:55, 1400.95it/s]

  0%|          | 1286/919652 [00:00<10:06, 1514.14it/s]

  0%|          | 1460/919652 [00:01<09:43, 1574.73it/s]

  0%|          | 1637/919652 [00:01<09:24, 1626.14it/s]

  0%|          | 1805/919652 [00:01<09:19, 1641.08it/s]

  0%|          | 1977/919652 [00:01<09:11, 1663.34it/s]

  0%|          | 2162/919652 [00:01<08:55, 1713.47it/s]

  0%|          | 2347/919652 [00:01<08:43, 1751.20it/s]

  0%|          | 2533/919652 [00:01<08:35, 1780.62it/s]

  0%|          | 2713/919652 [00:01<08:33, 1785.31it/

  5%|▍         | 43675/919652 [00:26<08:19, 1753.45it/s]

  5%|▍         | 43860/919652 [00:26<08:13, 1773.58it/s]

  5%|▍         | 44039/919652 [00:26<08:13, 1773.56it/s]

  5%|▍         | 44217/919652 [00:26<08:22, 1741.95it/s]

  5%|▍         | 44392/919652 [00:27<08:25, 1732.18it/s]

  5%|▍         | 44570/919652 [00:27<08:21, 1744.29it/s]

  5%|▍         | 44747/919652 [00:27<08:19, 1750.32it/s]

  5%|▍         | 44923/919652 [00:27<08:19, 1751.42it/s]

  5%|▍         | 45099/919652 [00:27<08:19, 1749.18it/s]

  5%|▍         | 45279/919652 [00:27<08:15, 1763.57it/s]

  5%|▍         | 45456/919652 [00:27<08:16, 1760.46it/s]

  5%|▍         | 45633/919652 [00:27<08:20, 1746.44it/s]

  5%|▍         | 45812/919652 [00:27<08:17, 1756.81it/s]

  5%|▌         | 45988/919652 [00:27<08:40, 1677.57it/s]

  5%|▌         | 46157/919652 [00:28<10:39, 1366.80it/s]

  5%|▌         | 46304/919652 [00:28<11:01, 1320.09it/s]

  5%|▌         | 46444/919652 [00:28<11:15, 1292.67it/s]

  5%|▌        

  9%|▉         | 84579/919652 [00:53<09:26, 1474.23it/s]

  9%|▉         | 84728/919652 [00:53<09:42, 1434.45it/s]

  9%|▉         | 84888/919652 [00:53<09:24, 1479.73it/s]

  9%|▉         | 85052/919652 [00:53<09:07, 1523.43it/s]

  9%|▉         | 85206/919652 [00:54<09:09, 1519.06it/s]

  9%|▉         | 85366/919652 [00:54<09:01, 1540.73it/s]

  9%|▉         | 85521/919652 [00:54<09:08, 1519.90it/s]

  9%|▉         | 85674/919652 [00:54<09:18, 1494.08it/s]

  9%|▉         | 85824/919652 [00:54<09:18, 1492.71it/s]

  9%|▉         | 85977/919652 [00:54<09:14, 1503.17it/s]

  9%|▉         | 86132/919652 [00:54<09:10, 1515.32it/s]

  9%|▉         | 86285/919652 [00:54<09:09, 1517.41it/s]

  9%|▉         | 86441/919652 [00:54<09:05, 1527.91it/s]

  9%|▉         | 86594/919652 [00:54<09:11, 1511.76it/s]

  9%|▉         | 86759/919652 [00:55<08:57, 1550.35it/s]

  9%|▉         | 86920/919652 [00:55<08:51, 1566.77it/s]

  9%|▉         | 87077/919652 [00:55<08:56, 1551.96it/s]

  9%|▉        

 14%|█▎        | 124263/919652 [01:20<10:43, 1236.98it/s]

 14%|█▎        | 124388/919652 [01:20<10:45, 1231.40it/s]

 14%|█▎        | 124527/919652 [01:20<10:24, 1273.67it/s]

 14%|█▎        | 124682/919652 [01:20<09:50, 1345.21it/s]

 14%|█▎        | 124832/919652 [01:20<09:32, 1387.85it/s]

 14%|█▎        | 124984/919652 [01:20<09:18, 1423.95it/s]

 14%|█▎        | 125128/919652 [01:20<09:54, 1337.41it/s]

 14%|█▎        | 125264/919652 [01:20<10:37, 1246.32it/s]

 14%|█▎        | 125392/919652 [01:20<10:42, 1237.01it/s]

 14%|█▎        | 125518/919652 [01:21<10:52, 1216.15it/s]

 14%|█▎        | 125646/919652 [01:21<10:43, 1233.75it/s]

 14%|█▎        | 125771/919652 [01:21<10:43, 1234.59it/s]

 14%|█▎        | 125908/919652 [01:21<10:24, 1271.70it/s]

 14%|█▎        | 126069/919652 [01:21<10:09, 1301.68it/s]

 14%|█▎        | 126231/919652 [01:21<09:33, 1382.99it/s]

 14%|█▎        | 126405/919652 [01:21<08:58, 1473.29it/s]

 14%|█▍        | 126570/919652 [01:21<08:41, 1521.65it/s

 18%|█▊        | 166202/919652 [01:46<07:12, 1740.96it/s]

 18%|█▊        | 166379/919652 [01:46<07:17, 1721.17it/s]

 18%|█▊        | 166557/919652 [01:46<07:13, 1737.97it/s]

 18%|█▊        | 166732/919652 [01:46<07:13, 1738.04it/s]

 18%|█▊        | 166907/919652 [01:46<07:12, 1740.84it/s]

 18%|█▊        | 167086/919652 [01:46<07:09, 1753.49it/s]

 18%|█▊        | 167262/919652 [01:46<07:12, 1739.78it/s]

 18%|█▊        | 167437/919652 [01:46<07:14, 1729.30it/s]

 18%|█▊        | 167611/919652 [01:47<07:28, 1678.62it/s]

 18%|█▊        | 167795/919652 [01:47<07:16, 1723.03it/s]

 18%|█▊        | 167971/919652 [01:47<07:13, 1733.33it/s]

 18%|█▊        | 168145/919652 [01:47<07:13, 1734.34it/s]

 18%|█▊        | 168319/919652 [01:47<07:40, 1630.72it/s]

 18%|█▊        | 168494/919652 [01:47<07:31, 1663.94it/s]

 18%|█▊        | 168666/919652 [01:47<07:26, 1680.17it/s]

 18%|█▊        | 168847/919652 [01:47<07:17, 1714.95it/s]

 18%|█▊        | 169033/919652 [01:47<07:07, 1754.26it/s

...

 88%|████████▊ | 810508/919652 [08:17<01:09, 1579.56it/s]

 88%|████████▊ | 810682/919652 [08:17<01:07, 1622.61it/s]

 88%|████████▊ | 810852/919652 [08:17<01:06, 1643.22it/s]

 88%|████████▊ | 811023/919652 [08:17<01:05, 1661.48it/s]

 88%|████████▊ | 811195/919652 [08:17<01:04, 1676.61it/s]

 88%|████████▊ | 811363/919652 [08:17<01:04, 1670.83it/s]

 88%|████████▊ | 811531/919652 [08:17<01:05, 1649.32it/s]

 88%|████████▊ | 811697/919652 [08:17<01:05, 1649.29it/s]

 88%|████████▊ | 811869/919652 [08:17<01:04, 1669.37it/s]

 88%|████████▊ | 812037/919652 [08:18<01:06, 1611.85it/s]

 88%|████████▊ | 812199/919652 [08:18<01:06, 1608.22it/s]

 88%|████████▊ | 812362/919652 [08:18<01:06, 1613.83it/s]

 88%|████████▊ | 812528/919652 [08:18<01:05, 1626.18it/s]

 88%|████████▊ | 812696/919652 [08:18<01:05, 1639.69it/s]

 88%|████████▊ | 812861/919652 [08:18<01:05, 1619.41it/s]

 88%|████████▊ | 813024/919652 [08:18<01:06, 1602.32it/s]

 88%|████████▊ | 813186/919652 [08:18<01:06, 1606.48it/s

 93%|█████████▎| 852310/919652 [08:42<00:41, 1630.37it/s]

 93%|█████████▎| 852474/919652 [08:43<00:42, 1569.65it/s]

 93%|█████████▎| 852632/919652 [08:43<00:43, 1547.41it/s]

 93%|█████████▎| 852801/919652 [08:43<00:42, 1586.42it/s]

 93%|█████████▎| 852961/919652 [08:43<00:42, 1578.45it/s]

 93%|█████████▎| 853120/919652 [08:43<00:42, 1569.35it/s]

 93%|█████████▎| 853301/919652 [08:43<00:40, 1633.94it/s]

 93%|█████████▎| 853468/919652 [08:43<00:40, 1644.18it/s]

 93%|█████████▎| 853634/919652 [08:43<00:41, 1594.89it/s]

 93%|█████████▎| 853797/919652 [08:43<00:41, 1604.36it/s]

 93%|█████████▎| 853964/919652 [08:44<00:40, 1622.07it/s]

 93%|█████████▎| 854130/919652 [08:44<00:40, 1632.80it/s]

 93%|█████████▎| 854299/919652 [08:44<00:39, 1648.08it/s]

 93%|█████████▎| 854465/919652 [08:44<00:39, 1635.65it/s]

 93%|█████████▎| 854633/919652 [08:44<00:39, 1647.10it/s]

 93%|█████████▎| 854798/919652 [08:44<00:39, 1627.31it/s]

 93%|█████████▎| 854961/919652 [08:44<00:40, 1610.40it/s

 97%|█████████▋| 893899/919652 [09:09<00:22, 1165.72it/s]

 97%|█████████▋| 894029/919652 [09:09<00:21, 1200.21it/s]

 97%|█████████▋| 894153/919652 [09:09<00:23, 1076.65it/s]

 97%|█████████▋| 894272/919652 [09:09<00:22, 1107.67it/s]

 97%|█████████▋| 894396/919652 [09:09<00:22, 1143.41it/s]

 97%|█████████▋| 894522/919652 [09:09<00:21, 1173.13it/s]

 97%|█████████▋| 894695/919652 [09:09<00:19, 1297.75it/s]

 97%|█████████▋| 894844/919652 [09:09<00:18, 1349.16it/s]

 97%|█████████▋| 894984/919652 [09:09<00:18, 1306.32it/s]

 97%|█████████▋| 895119/919652 [09:10<00:19, 1252.05it/s]

 97%|█████████▋| 895248/919652 [09:10<00:19, 1233.56it/s]

 97%|█████████▋| 895374/919652 [09:10<00:20, 1193.89it/s]

 97%|█████████▋| 895496/919652 [09:10<00:20, 1197.83it/s]

 97%|█████████▋| 895617/919652 [09:10<00:20, 1173.08it/s]

 97%|█████████▋| 895736/919652 [09:10<00:20, 1177.96it/s]

 97%|█████████▋| 895855/919652 [09:10<00:20, 1177.09it/s]

 97%|█████████▋| 895974/919652 [09:10<00:20, 1167.36it/s

--- 9 minutes ---


['доп карта', 'день карта заказать день выпустить']

In [299]:
output_data['sentences']=sentences

сохраним исходные данные вместе с отпрепроцешенными сообщениями

In [301]:
output_data.to_csv('data_history.csv')

------------------------------------------------------------------------------------------------------------------------------

# 2. ЗАГРУЗИМ УЖЕ ПОДГОТОВЛЕННУЮ ВЫГРУЗКУ

##### токенизируем, убрав все символы и приведя к нижнему регистру

In [302]:
data_tmp = output_data

In [282]:
test_data_tnmp = data_tmp[(data_tmp.isna()==False)]
test_data_tnmp.shape

(1779750, 23)

In [303]:
sequences_tmp=[]
for phrase in trange(data_tmp.shape[0]):
    sequences_tmp.append(text_to_word_sequence(str(data_tmp.sentences.iloc[phrase])))



  0%|          | 0/3974676 [00:00<?, ?it/s]

  0%|          | 1741/3974676 [00:00<03:48, 17401.93it/s]

  0%|          | 3637/3974676 [00:00<03:42, 17841.65it/s]

  0%|          | 5508/3974676 [00:00<03:39, 18092.48it/s]

  0%|          | 7337/3974676 [00:00<03:38, 18149.04it/s]

  0%|          | 9204/3974676 [00:00<03:36, 18302.01it/s]

  0%|          | 10744/3974676 [00:00<03:56, 16732.91it/s]

  0%|          | 12335/3974676 [00:00<04:00, 16475.59it/s]

  0%|          | 14098/3974676 [00:00<03:55, 16804.05it/s]

  0%|          | 15859/3974676 [00:00<03:52, 17037.23it/s]

  0%|          | 17694/3974676 [00:01<03:47, 17409.02it/s]

  0%|          | 19552/3974676 [00:01<03:42, 17743.60it/s]

  1%|          | 21409/3974676 [00:01<03:39, 17982.68it/s]

  1%|          | 23241/3974676 [00:01<03:38, 18081.07it/s]

  1%|          | 25038/3974676 [00:01<03:41, 17792.60it/s]

  1%|          | 27224/3974676 [00:01<03:29, 18842.74it/s]

  1%|          | 29286/3974676 [00:01<03:23, 19342.30it/s]

 14%|█▎        | 545637/3974676 [00:32<02:54, 19676.54it/s]

 14%|█▍        | 547754/3974676 [00:32<02:50, 20100.94it/s]

 14%|█▍        | 549923/3974676 [00:32<02:46, 20551.27it/s]

 14%|█▍        | 552101/3974676 [00:32<02:43, 20904.76it/s]

 14%|█▍        | 554392/3974676 [00:32<02:39, 21466.00it/s]

 14%|█▍        | 556552/3974676 [00:33<02:38, 21502.92it/s]

 14%|█▍        | 558712/3974676 [00:33<02:39, 21482.61it/s]

 14%|█▍        | 560867/3974676 [00:33<02:40, 21211.79it/s]

 14%|█▍        | 563111/3974676 [00:33<02:38, 21564.38it/s]

 14%|█▍        | 565326/3974676 [00:33<02:36, 21734.77it/s]

 14%|█▍        | 567523/3974676 [00:33<02:36, 21804.58it/s]

 14%|█▍        | 569707/3974676 [00:33<02:36, 21733.57it/s]

 14%|█▍        | 571900/3974676 [00:33<02:36, 21791.70it/s]

 14%|█▍        | 574108/3974676 [00:33<02:35, 21876.57it/s]

 14%|█▍        | 576308/3974676 [00:33<02:35, 21912.27it/s]

 15%|█▍        | 578500/3974676 [00:34<02:39, 21292.51it/s]

 15%|█▍        | 580634/

 28%|██▊       | 1096624/3974676 [00:59<02:37, 18276.91it/s]

 28%|██▊       | 1098550/3974676 [00:59<02:41, 17838.70it/s]

 28%|██▊       | 1100405/3974676 [00:59<02:41, 17743.67it/s]

 28%|██▊       | 1102229/3974676 [00:59<02:42, 17706.03it/s]

 28%|██▊       | 1104035/3974676 [00:59<02:41, 17750.68it/s]

 28%|██▊       | 1105835/3974676 [00:59<02:44, 17475.32it/s]

 28%|██▊       | 1107661/3974676 [00:59<02:41, 17703.15it/s]

 28%|██▊       | 1109532/3974676 [00:59<02:39, 17993.19it/s]

 28%|██▊       | 1111806/3974676 [01:00<02:29, 19193.42it/s]

 28%|██▊       | 1113798/3974676 [01:00<02:27, 19404.11it/s]

 28%|██▊       | 1115949/3974676 [01:00<02:23, 19991.08it/s]

 28%|██▊       | 1118108/3974676 [01:00<02:19, 20443.35it/s]

 28%|██▊       | 1120253/3974676 [01:00<02:18, 20634.40it/s]

 28%|██▊       | 1122423/3974676 [01:00<02:16, 20942.65it/s]

 28%|██▊       | 1124551/3974676 [01:00<02:15, 21040.60it/s]

 28%|██▊       | 1126733/3974676 [01:00<02:13, 21268.21it/s]

 28%|██▊

 41%|████      | 1638752/3974676 [01:25<02:08, 18141.16it/s]

 41%|████▏     | 1640580/3974676 [01:25<02:09, 18034.48it/s]

 41%|████▏     | 1642393/3974676 [01:26<02:10, 17908.68it/s]

 41%|████▏     | 1644517/3974676 [01:26<02:03, 18792.75it/s]

 41%|████▏     | 1646836/3974676 [01:26<01:56, 19925.51it/s]

 41%|████▏     | 1649200/3974676 [01:26<01:51, 20909.79it/s]

 42%|████▏     | 1651602/3974676 [01:26<01:46, 21753.48it/s]

 42%|████▏     | 1653941/3974676 [01:26<01:44, 22218.20it/s]

 42%|████▏     | 1656188/3974676 [01:26<01:44, 22287.87it/s]

 42%|████▏     | 1658454/3974676 [01:26<01:43, 22397.48it/s]

 42%|████▏     | 1660789/3974676 [01:26<01:42, 22672.78it/s]

 42%|████▏     | 1663157/3974676 [01:26<01:40, 22964.61it/s]

 42%|████▏     | 1665463/3974676 [01:27<01:40, 22993.10it/s]

 42%|████▏     | 1667856/3974676 [01:27<01:39, 23265.32it/s]

 42%|████▏     | 1670279/3974676 [01:27<01:37, 23546.20it/s]

 42%|████▏     | 1672638/3974676 [01:27<01:38, 23467.47it/s]

 42%|███

 55%|█████▌    | 2195394/3974676 [01:52<01:08, 25802.71it/s]

 55%|█████▌    | 2197992/3974676 [01:52<01:08, 25854.33it/s]

 55%|█████▌    | 2200578/3974676 [01:52<01:09, 25468.50it/s]

 55%|█████▌    | 2203127/3974676 [01:53<01:10, 25260.46it/s]

 55%|█████▌    | 2205655/3974676 [01:53<01:10, 25129.24it/s]

 56%|█████▌    | 2208263/3974676 [01:53<01:09, 25404.74it/s]

 56%|█████▌    | 2210842/3974676 [01:53<01:09, 25518.21it/s]

 56%|█████▌    | 2213500/3974676 [01:53<01:08, 25817.65it/s]

 56%|█████▌    | 2216084/3974676 [01:53<01:08, 25727.77it/s]

 56%|█████▌    | 2218735/3974676 [01:53<01:07, 25956.52it/s]

 56%|█████▌    | 2221373/3974676 [01:53<01:07, 26080.72it/s]

 56%|█████▌    | 2224010/3974676 [01:53<01:06, 26165.52it/s]

 56%|█████▌    | 2226687/3974676 [01:53<01:06, 26342.49it/s]

 56%|█████▌    | 2229348/3974676 [01:54<01:06, 26422.07it/s]

 56%|█████▌    | 2231991/3974676 [01:54<01:07, 25894.38it/s]

 56%|█████▌    | 2234584/3974676 [01:54<01:07, 25710.54it/s]

 56%|███

 70%|███████   | 2786518/3974676 [02:19<00:45, 25921.42it/s]

 70%|███████   | 2789112/3974676 [02:19<00:47, 25012.23it/s]

 70%|███████   | 2791621/3974676 [02:19<00:51, 22966.15it/s]

 70%|███████   | 2794170/3974676 [02:19<00:49, 23668.49it/s]

 70%|███████   | 2796569/3974676 [02:19<00:49, 23704.22it/s]

 70%|███████   | 2799188/3974676 [02:19<00:48, 24397.34it/s]

 70%|███████   | 2801817/3974676 [02:20<00:47, 24935.68it/s]

 71%|███████   | 2804439/3974676 [02:20<00:46, 25307.22it/s]

 71%|███████   | 2807097/3974676 [02:20<00:45, 25673.84it/s]

 71%|███████   | 2809676/3974676 [02:20<00:46, 25315.87it/s]

 71%|███████   | 2812276/3974676 [02:20<00:45, 25513.99it/s]

 71%|███████   | 2814868/3974676 [02:20<00:45, 25632.72it/s]

 71%|███████   | 2817481/3974676 [02:20<00:44, 25779.40it/s]

 71%|███████   | 2820063/3974676 [02:20<00:44, 25705.35it/s]

 71%|███████   | 2822636/3974676 [02:20<00:44, 25692.34it/s]

 71%|███████   | 2825270/3974676 [02:20<00:44, 25882.72it/s]

 71%|███

 85%|████████▌ | 3388813/3974676 [02:47<00:21, 27261.74it/s]

 85%|████████▌ | 3391553/3974676 [02:47<00:21, 27248.00it/s]

 85%|████████▌ | 3394287/3974676 [02:48<00:21, 26686.40it/s]

 85%|████████▌ | 3397058/3974676 [02:48<00:21, 26983.71it/s]

 86%|████████▌ | 3399866/3974676 [02:48<00:21, 27301.63it/s]

 86%|████████▌ | 3402701/3974676 [02:48<00:20, 27607.86it/s]

 86%|████████▌ | 3405591/3974676 [02:48<00:20, 27980.98it/s]

 86%|████████▌ | 3408394/3974676 [02:48<00:24, 23426.97it/s]

 86%|████████▌ | 3410866/3974676 [02:48<00:24, 23020.44it/s]

 86%|████████▌ | 3413260/3974676 [02:48<00:24, 22618.70it/s]

 86%|████████▌ | 3415588/3974676 [02:48<00:25, 22100.93it/s]

 86%|████████▌ | 3417847/3974676 [02:49<00:25, 22072.68it/s]

 86%|████████▌ | 3420089/3974676 [02:49<00:25, 22064.35it/s]

 86%|████████▌ | 3422320/3974676 [02:49<00:25, 21886.57it/s]

 86%|████████▌ | 3424770/3974676 [02:49<00:24, 22608.46it/s]

 86%|████████▌ | 3427626/3974676 [02:49<00:22, 24115.57it/s]

 86%|███

------------------------------------------------------------------------------------------------------------------------------

### обучим или загрузим готовую модель word2vec

In [304]:
from gensim.models import Word2Vec

In [305]:
%%time
wvec=Word2Vec(sequences_tmp, window=7, size=300, min_count=1, workers = 12)

CPU times: user 16min 4s, sys: 35.5 s, total: 16min 39s
Wall time: 4min 48s


In [ ]:
# wvec.save('word2vec_normalized.hdf5')
# wvec = Word2Vec.load('word2vec_normalized.hdf5')

------------------------------------------------------------------------------------------------------------------------------

##### создадим вектор для каждой отпрепроцешенной фразы

In [306]:
vectors_tmp = []
for phrase in trange(len(sequences_tmp)):
    temp_vector = []
    if sequences_tmp[phrase]:
        for word in sequences_tmp[phrase]:
            if word in wvec:
                temp_vector.append(wvec.wv.__getitem__(word))
            else:
                temp_vector.append(np.zeros(300))
        vectors_tmp.append(np.mean(temp_vector, axis=0))
    else:
        vectors_tmp.append(np.zeros(300))



  0%|          | 0/3974676 [00:00<?, ?it/s]

  0%|          | 779/3974676 [00:00<08:30, 7788.34it/s]

  0%|          | 1765/3974676 [00:00<07:58, 8310.56it/s]

  0%|          | 2817/3974676 [00:00<07:27, 8869.00it/s]

  0%|          | 3875/3974676 [00:00<07:06, 9319.35it/s]

  0%|          | 4900/3974676 [00:00<06:54, 9579.03it/s]

  0%|          | 6033/3974676 [00:00<06:35, 10042.63it/s]

  0%|          | 7144/3974676 [00:00<06:23, 10339.52it/s]

  0%|          | 8144/3974676 [00:00<06:27, 10233.92it/s]

  0%|          | 9216/3974676 [00:00<06:22, 10374.36it/s]

  0%|          | 10311/3974676 [00:01<06:16, 10538.03it/s]

  0%|          | 11353/3974676 [00:01<06:17, 10499.13it/s]

  0%|          | 12406/3974676 [00:01<06:17, 10506.25it/s]

  0%|          | 13506/3974676 [00:01<06:11, 10649.12it/s]

  0%|          | 14595/3974676 [00:01<06:09, 10718.66it/s]

  0%|          | 15686/3974676 [00:01<06:07, 10775.09it/s]

  0%|          | 16762/3974676 [00:01<06:13, 10593.41it/s]

  0%|   

  6%|▌         | 247571/3974676 [00:25<06:13, 9986.96it/s]

  6%|▋         | 248615/3974676 [00:25<06:08, 10114.62it/s]

  6%|▋         | 249646/3974676 [00:25<06:06, 10172.24it/s]

  6%|▋         | 250665/3974676 [00:26<06:08, 10099.40it/s]

  6%|▋         | 251685/3974676 [00:26<06:07, 10127.88it/s]

  6%|▋         | 252699/3974676 [00:26<06:14, 9925.37it/s] 

  6%|▋         | 253718/3974676 [00:26<06:12, 9999.01it/s]

  6%|▋         | 254719/3974676 [00:26<06:13, 9963.34it/s]

  6%|▋         | 255753/3974676 [00:26<06:09, 10068.82it/s]

  6%|▋         | 256784/3974676 [00:26<06:06, 10138.76it/s]

  6%|▋         | 257831/3974676 [00:26<06:03, 10235.54it/s]

  7%|▋         | 258856/3974676 [00:26<06:24, 9674.80it/s] 

  7%|▋         | 259841/3974676 [00:26<06:22, 9723.14it/s]

  7%|▋         | 260910/3974676 [00:27<06:11, 9992.52it/s]

  7%|▋         | 261959/3974676 [00:27<06:06, 10131.32it/s]

  7%|▋         | 262996/3974676 [00:27<06:03, 10199.43it/s]

  7%|▋         | 264019/39746

 12%|█▏        | 491238/3974676 [00:50<06:32, 8867.88it/s]

 12%|█▏        | 492126/3974676 [00:50<06:37, 8754.10it/s]

 12%|█▏        | 493027/3974676 [00:51<06:34, 8827.54it/s]

 12%|█▏        | 493929/3974676 [00:51<06:31, 8882.24it/s]

 12%|█▏        | 494829/3974676 [00:51<06:30, 8914.00it/s]

 12%|█▏        | 495721/3974676 [00:51<06:32, 8858.51it/s]

 12%|█▏        | 496629/3974676 [00:51<06:30, 8909.59it/s]

 13%|█▎        | 497557/3974676 [00:51<06:37, 8752.26it/s]

 13%|█▎        | 498440/3974676 [00:51<06:36, 8770.95it/s]

 13%|█▎        | 499344/3974676 [00:51<06:32, 8849.71it/s]

 13%|█▎        | 500267/3974676 [00:51<06:27, 8960.08it/s]

 13%|█▎        | 501167/3974676 [00:52<06:27, 8971.55it/s]

 13%|█▎        | 502118/3974676 [00:52<06:20, 9126.30it/s]

 13%|█▎        | 503032/3974676 [00:52<06:20, 9121.60it/s]

 13%|█▎        | 503945/3974676 [00:52<06:22, 9081.42it/s]

 13%|█▎        | 504854/3974676 [00:52<06:25, 9002.08it/s]

 13%|█▎        | 505755/3974676 [00:52<0

 18%|█▊        | 721822/3974676 [01:16<05:40, 9547.37it/s]

 18%|█▊        | 722821/3974676 [01:16<05:36, 9675.33it/s]

 18%|█▊        | 723810/3974676 [01:16<05:33, 9737.04it/s]

 18%|█▊        | 724785/3974676 [01:16<05:40, 9536.43it/s]

 18%|█▊        | 725742/3974676 [01:16<05:40, 9545.80it/s]

 18%|█▊        | 726698/3974676 [01:16<05:47, 9341.84it/s]

 18%|█▊        | 727667/3974676 [01:16<05:43, 9441.97it/s]

 18%|█▊        | 728658/3974676 [01:17<05:38, 9576.83it/s]

 18%|█▊        | 729625/3974676 [01:17<05:37, 9603.48it/s]

 18%|█▊        | 730587/3974676 [01:17<05:46, 9361.56it/s]

 18%|█▊        | 731539/3974676 [01:17<05:44, 9405.99it/s]

 18%|█▊        | 732500/3974676 [01:17<05:42, 9466.17it/s]

 18%|█▊        | 733448/3974676 [01:17<05:44, 9420.86it/s]

 18%|█▊        | 734391/3974676 [01:17<05:46, 9338.34it/s]

 19%|█▊        | 735331/3974676 [01:17<05:46, 9356.71it/s]

 19%|█▊        | 736346/3974676 [01:17<05:38, 9577.56it/s]

 19%|█▊        | 737327/3974676 [01:17<0

 21%|██        | 840827/3974676 [01:29<06:08, 8501.23it/s]

 21%|██        | 841681/3974676 [01:29<06:13, 8388.32it/s]

 21%|██        | 842555/3974676 [01:29<06:08, 8488.86it/s]

 21%|██        | 843406/3974676 [01:29<06:12, 8400.11it/s]

 21%|██        | 844248/3974676 [01:29<06:16, 8324.32it/s]

 21%|██▏       | 845104/3974676 [01:29<06:12, 8393.27it/s]

 21%|██▏       | 846018/3974676 [01:29<06:03, 8602.96it/s]

 21%|██▏       | 846923/3974676 [01:29<05:58, 8731.90it/s]

 21%|██▏       | 847799/3974676 [01:30<05:59, 8696.79it/s]

 21%|██▏       | 848671/3974676 [01:30<06:04, 8585.42it/s]

 21%|██▏       | 849593/3974676 [01:30<05:56, 8759.22it/s]

 21%|██▏       | 850471/3974676 [01:30<05:56, 8762.17it/s]

 21%|██▏       | 851349/3974676 [01:30<05:57, 8726.92it/s]

 21%|██▏       | 852236/3974676 [01:30<05:56, 8768.38it/s]

 21%|██▏       | 853114/3974676 [01:30<05:56, 8756.10it/s]

 21%|██▏       | 854006/3974676 [01:30<05:54, 8800.54it/s]

 22%|██▏       | 854887/3974676 [01:30<0

...

 64%|██████▎   | 2525473/3974676 [04:24<02:05, 11572.72it/s]

 64%|██████▎   | 2526632/3974676 [04:24<02:06, 11455.43it/s]

 64%|██████▎   | 2527804/3974676 [04:24<02:05, 11530.95it/s]

 64%|██████▎   | 2528959/3974676 [04:24<02:06, 11462.56it/s]

 64%|██████▎   | 2530117/3974676 [04:24<02:05, 11495.11it/s]

 64%|██████▎   | 2531303/3974676 [04:24<02:04, 11601.97it/s]

 64%|██████▎   | 2532505/3974676 [04:24<02:03, 11717.51it/s]

 64%|██████▎   | 2533678/3974676 [04:25<02:03, 11666.57it/s]

 64%|██████▍   | 2534846/3974676 [04:25<02:04, 11601.13it/s]

 64%|██████▍   | 2536102/3974676 [04:25<02:01, 11872.52it/s]

 64%|██████▍   | 2537297/3974676 [04:25<02:00, 11894.01it/s]

 64%|██████▍   | 2538488/3974676 [04:25<02:02, 11770.93it/s]

 64%|██████▍   | 2539667/3974676 [04:25<02:01, 11763.87it/s]

 64%|██████▍   | 2540845/3974676 [04:25<02:03, 11590.13it/s]

 64%|██████▍   | 2542006/3974676 [04:25<02:03, 11560.58it/s]

 64%|██████▍   | 2543198/3974676 [04:25<02:02, 11664.61it/s]

 64%|███

 70%|███████   | 2791855/3974676 [04:49<01:48, 10874.49it/s]

 70%|███████   | 2793004/3974676 [04:49<01:46, 11048.31it/s]

 70%|███████   | 2794111/3974676 [04:49<01:49, 10821.06it/s]

 70%|███████   | 2795196/3974676 [04:49<01:50, 10682.57it/s]

 70%|███████   | 2796313/3974676 [04:49<01:48, 10823.63it/s]

 70%|███████   | 2797461/3974676 [04:49<01:46, 11009.24it/s]

 70%|███████   | 2798636/3974676 [04:49<01:44, 11220.51it/s]

 70%|███████   | 2799766/3974676 [04:49<01:44, 11244.14it/s]

 70%|███████   | 2800893/3974676 [04:49<01:44, 11217.56it/s]

 70%|███████   | 2802035/3974676 [04:50<01:43, 11275.45it/s]

 71%|███████   | 2803166/3974676 [04:50<01:43, 11284.79it/s]

 71%|███████   | 2804309/3974676 [04:50<01:43, 11326.48it/s]

 71%|███████   | 2805443/3974676 [04:50<01:46, 11011.74it/s]

 71%|███████   | 2806575/3974676 [04:50<01:45, 11099.79it/s]

 71%|███████   | 2807691/3974676 [04:50<01:44, 11116.46it/s]

 71%|███████   | 2808824/3974676 [04:50<01:44, 11177.72it/s]

 71%|███

 77%|███████▋  | 3047559/3974676 [05:13<01:31, 10123.68it/s]

 77%|███████▋  | 3048573/3974676 [05:14<01:31, 10113.97it/s]

 77%|███████▋  | 3049592/3974676 [05:14<01:31, 10135.65it/s]

 77%|███████▋  | 3050658/3974676 [05:14<01:29, 10286.91it/s]

 77%|███████▋  | 3051725/3974676 [05:14<01:28, 10397.19it/s]

 77%|███████▋  | 3052785/3974676 [05:14<01:28, 10457.03it/s]

 77%|███████▋  | 3053832/3974676 [05:14<01:28, 10411.29it/s]

 77%|███████▋  | 3054874/3974676 [05:14<01:29, 10267.39it/s]

 77%|███████▋  | 3056417/3974676 [05:14<01:20, 11411.98it/s]

 77%|███████▋  | 3057943/3974676 [05:14<01:14, 12345.29it/s]

 77%|███████▋  | 3059394/3974676 [05:14<01:10, 12922.91it/s]

 77%|███████▋  | 3060909/3974676 [05:15<01:07, 13518.46it/s]

 77%|███████▋  | 3062420/3974676 [05:15<01:05, 13957.46it/s]

 77%|███████▋  | 3063908/3974676 [05:15<01:04, 14220.43it/s]

 77%|███████▋  | 3065431/3974676 [05:15<01:02, 14507.49it/s]

 77%|███████▋  | 3066968/3974676 [05:15<01:01, 14755.01it/s]

 77%|███

 86%|████████▌ | 3413025/3974676 [05:38<00:33, 16589.77it/s]

 86%|████████▌ | 3414727/3974676 [05:38<00:33, 16714.69it/s]

 86%|████████▌ | 3416473/3974676 [05:38<00:32, 16929.51it/s]

 86%|████████▌ | 3418168/3974676 [05:38<00:33, 16762.77it/s]

 86%|████████▌ | 3419940/3974676 [05:39<00:32, 17036.32it/s]

 86%|████████▌ | 3421670/3974676 [05:39<00:32, 17114.26it/s]

 86%|████████▌ | 3423384/3974676 [05:39<00:33, 16641.80it/s]

 86%|████████▌ | 3425083/3974676 [05:39<00:32, 16742.03it/s]

 86%|████████▌ | 3426802/3974676 [05:39<00:32, 16870.36it/s]

 86%|████████▋ | 3428492/3974676 [05:39<00:32, 16753.95it/s]

 86%|████████▋ | 3430170/3974676 [05:39<00:33, 16310.46it/s]

 86%|████████▋ | 3431805/3974676 [05:39<00:33, 16176.71it/s]

 86%|████████▋ | 3433443/3974676 [05:39<00:33, 16236.74it/s]

 86%|████████▋ | 3435093/3974676 [05:39<00:33, 16314.15it/s]

 86%|████████▋ | 3436779/3974676 [05:40<00:32, 16474.09it/s]

 87%|████████▋ | 3438428/3974676 [05:40<00:33, 16004.89it/s]

 87%|███

 96%|█████████▌| 3801563/3974676 [06:03<00:11, 14675.71it/s]

 96%|█████████▌| 3803140/3974676 [06:03<00:11, 14984.85it/s]

 96%|█████████▌| 3804643/3974676 [06:03<00:11, 14834.11it/s]

 96%|█████████▌| 3806130/3974676 [06:03<00:11, 14781.33it/s]

 96%|█████████▌| 3807616/3974676 [06:03<00:11, 14789.18it/s]

 96%|█████████▌| 3809097/3974676 [06:03<00:11, 14634.83it/s]

 96%|█████████▌| 3810562/3974676 [06:03<00:11, 14608.04it/s]

 96%|█████████▌| 3812059/3974676 [06:03<00:11, 14714.17it/s]

 96%|█████████▌| 3813532/3974676 [06:04<00:10, 14712.79it/s]

 96%|█████████▌| 3815004/3974676 [06:04<00:10, 14617.48it/s]

 96%|█████████▌| 3816467/3974676 [06:04<00:10, 14469.45it/s]

 96%|█████████▌| 3817915/3974676 [06:04<00:10, 14385.19it/s]

 96%|█████████▌| 3819355/3974676 [06:04<00:10, 14312.01it/s]

 96%|█████████▌| 3820787/3974676 [06:04<00:10, 14168.02it/s]

 96%|█████████▌| 3822205/3974676 [06:04<00:10, 14162.31it/s]

 96%|█████████▌| 3823622/3974676 [06:04<00:10, 14126.52it/s]

 96%|███

In [307]:
# функция принимает предобработанную фразу, токенизирует и возвращает вектор
def vectorize(phrase):
#     phrase = str(phrase).split(' ')
    temp_vect=[]
    if len(phrase)>0:
#         phrase = preprocess_text(str(phrase)).split(' ')
        for word in phrase:
            if word in wvec:
                temp_vect.append(wvec.wv.__getitem__(word))
            else:
                temp_vect.append(np.zeros(300))
        return np.mean(temp_vect, axis=0)
    else:
        return np.asarray(np.zeros(300))

------------------------------------------------------------------------------------------------------------------------------

### обучим модель, предварительно указав количество ближайших опорных фраз для поиска

PS: чем больше фраз на одно сообщение, тем больше времени на обработку

In [346]:
n_neighbors = 20

In [347]:
from sklearn.neighbors import NearestNeighbors

In [348]:
start_time = time.time()

from sklearn.neighbors import NearestNeighbors
neigh=NearestNeighbors(n_neighbors=n_neighbors+1, n_jobs=1, metric='cosine', algorithm='brute')
neigh.fit(vectors_tmp)

print("--- %s minutes ---" % (round((time.time() - start_time)/60)))

--- 0 minutes ---


-------------------------------------------------------------------------------------------------------------------------------

### предполагает подгрузку файла с отмеченными "1" значениями в поле "ФЛАГ_ПОИСК" для тех фраз, по которым нужен поиск

#### используем ту же книгу, к имени книги добавляем отметку "_updated" перед .xlsx, загружаем в эту же папку

In [371]:
data=pd.read_csv('data.txt', delimiter='\t', encoding='cp1251')

In [372]:
data['ФЛАГ_ПОИСК']=1

In [153]:
data.rename(columns={'preprocessed': 'message', 'Интент': 'intent'}, inplace=True)

In [373]:
message_col='message'#'опорная фраза'

### запишем функцию, возвращающую для переданных фраз k+1 ближайших соседей, обьединим результаты с исходным датафреймом

In [375]:
# определим фунцкцию поиска ближайших опорных фраз:
# на вход преобразованная строка, возвращает словарь
def find_neighbour_wvec(sentence):
    res_dict={}
    wvec_vector = vectorize(sentence.split(' '))

    res=neigh.kneighbors([wvec_vector], return_distance=True)
    for j in range(len(res[0][0])):
        res_dict.setdefault(res[1][0][j], res[0][0][j])
    return res_dict

In [377]:
%%time
data_filtered=data[data.ФЛАГ_ПОИСК==1]
# data_filtered = data.copy()
data_filtered['sentences']=None
message_col_ind=data_filtered.columns.tolist().index(message_col)

for i in trange(data_filtered.shape[0]):
    data_filtered.iloc[i, -1]=preprocess_text(data_filtered.iloc[i, message_col_ind])

In [378]:
start_time = time.time()

final_df=pd.DataFrame(columns=['исходные_сообщения', 'похожие_сообщения', 'дистанция', 'исходные_лемм_фразы', 'похожие_лемм_фразы', 'номер_строки_базы', 'интент'], index=None)
for i in trange(data_filtered.shape[0]):
    sentence = data_filtered.sentences.iloc[i]
    temp_dict=dict(sorted(find_neighbour_wvec(sentence).items(), key=lambda kv: kv[1])) # отсортированный по значению словарь
    temp_df=pd.DataFrame(columns=['исходные_сообщения', 'похожие_сообщения', 'дистанция', 'исходные_лемм_фразы', 'похожие_лемм_фразы', 'номер_строки_базы', 'интент'], index=None)
    temp_df.iloc[:, 1]=data_tmp.Сообщение.iloc[list(temp_dict.keys())].tolist()
    temp_df.iloc[:, 2]=list(temp_dict.values())
    temp_df.iloc[:, 3]=sentence
    temp_df.iloc[:, 4]=data_tmp.sentences.iloc[list(temp_dict.keys())].tolist()
    temp_df.iloc[:, 5]=list(temp_dict.keys())
    temp_df.iloc[:, 0]=data_filtered[message_col].iloc[i]
#     temp_df.iloc[:, 6]=data_filtered['название'].iloc[l]
    final_df=pd.concat([final_df, temp_df])
print("--- %s minutes ---" % (round((time.time() - start_time)/60)))
final_df.head()



  0%|          | 0/86 [00:00<?, ?it/s]

  1%|          | 1/86 [00:14<20:43, 14.64s/it]

  2%|▏         | 2/86 [00:30<21:03, 15.04s/it]

  3%|▎         | 3/86 [00:43<20:06, 14.53s/it]

  5%|▍         | 4/86 [00:57<19:38, 14.37s/it]

  6%|▌         | 5/86 [01:11<18:59, 14.07s/it]

  7%|▋         | 6/86 [01:22<17:42, 13.28s/it]

  8%|▊         | 7/86 [01:35<17:23, 13.21s/it]

  9%|▉         | 8/86 [01:47<16:35, 12.76s/it]

 10%|█         | 9/86 [02:00<16:21, 12.74s/it]

 12%|█▏        | 10/86 [02:11<15:40, 12.38s/it]

 13%|█▎        | 11/86 [02:23<15:06, 12.09s/it]

 14%|█▍        | 12/86 [02:34<14:44, 11.95s/it]

 15%|█▌        | 13/86 [02:47<14:50, 12.20s/it]

 16%|█▋        | 14/86 [02:58<14:20, 11.95s/it]

 17%|█▋        | 15/86 [03:10<13:54, 11.76s/it]

 19%|█▊        | 16/86 [03:21<13:34, 11.64s/it]

 20%|█▉        | 17/86 [03:34<13:52, 12.06s/it]

 21%|██        | 18/86 [03:45<13:25, 11.85s/it]

 22%|██▏       | 19/86 [03:57<13:02, 11.68s/it]

 23%|██▎       | 20/86 [04:10<13:20,

--- 17 minutes ---


,исходные_сообщения,похожие_сообщения,дистанция,исходные_лемм_фразы,похожие_лемм_фразы,номер_строки_базы,интент
0,есть ли открытые счета в банке ?,Есть ли открытые счета в банке,0.0,открытый счёт банка,открытый счёт банка,3694184,NaN
1,есть ли открытые счета в банке ?,Когда открыт счёт в банке?,0.0,открытый счёт банка,открытый счёт банка,3609979,NaN
2,есть ли открытые счета в банке ?,Есть ли у меня открытые счета в банке?,0.0,открытый счёт банка,открытый счёт банка,3154993,NaN
3,есть ли открытые счета в банке ?,когда был открыт счет в банке,0.0,открытый счёт банка,открытый счёт банка,3752074,NaN
4,есть ли открытые счета в банке ?,Открытые счета в банке,0.0,открытый счёт банка,открытый счёт банка,3207953,NaN


In [379]:
# запишем в excel
wb_name_final='похожие_фразы_'+dt.strftime(dt.today(),'%Y-%m-%d')+'_final.xlsx'
final_df.to_excel(wb_name_final, sheet_name='фразы')